<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Arbitrary Angle ZZ Gates via pytket

This notebook contains a comparison of circuits with and without use of Quantinuum's native arbitrary-angle ZZ gate in `pytket`. The circuit primitive, the Quantum Fourier Transform (QFT) is constructed with `pytket`. The inverse QFT is an important primitive used in the [Phase Estimation Algorithm (PEA)](https://tket.quantinuum.com/examples/phase_estimation.html). PEA is used to estimate the phase corresponding to the eigenvalue of a specified unitary.

Arbitrary-angle two-qubit gates can be used to improve fidelity of the output and to decrease two-qubit gate depth. Specifically, the error from arbitrary-angle two-qubit gates is less than the fixed-angle two-qubit gate for small angles. The error from both gates is the same at angle $\frac{\phi}{2}$. The error from arbitrary-angle two-qubit gates increases with angle size.

* [Arbitrary Angle ZZ Gates](#Arbitrary-Angle-ZZ-Gates)<br>
* [Quantum Fourier Transform](#Quantum-Fourier-Transform)<br>
* [QFT with Fixed Angle Gates](#QFT-with-Fixed-Angle-Gates)<br>
* [QFT with Arbitrary Angle ZZ Gates](#QFT-with-Arbitrary-Angle-ZZ-Gates)<br>
* [Compare Results](#Compare-Results)

## Arbitrary Angle ZZ Gates

Quantinuum System Model H1's native gate set includes arbitrary angle ZZ gates. This is beneficial for reducing the 2-qubit gate count for many quantum algorithms and gate sequences.

$$RZZ(\theta) = e^{-i\frac{\theta}{2}\hat{Z} \otimes \hat{Z}}= e^{-i \frac{\theta}{2}} \begin{bmatrix} 1 & 0 & 0 & 0\\ 0 & e^{-i\theta} & 0 & 0\\ 0 & 0 & e^{-i\theta} & 0\\ 0 & 0 & 0 & 1 \end{bmatrix}$$

Note that $RZZ(\frac{\pi}{2}) = ZZ()$.

Quantum circuits that use the gate sequence CNOT, RZ, CNOT can be replaced with the arbitrary angle ZZ gate, shown below. This enables a lower number of 2-qubit gates in a quantum circuit, improving performance by decreasing gate errors.

<br>

<div style="text-align: center;"><br>
<img src="rzz.png" width="250"/><br>
</div>

This notebook demonstrates the Quantum Fourier Transform (QFT) with and without the $RZZ$ gate.

## Quantum Fourier Transform

The Quantum Fourier Transform (QFT) is an algorithm that serves as a sub-routine in multiple quantum algorithms, including Shor's factoring algorithm. Below are two functions, written in `pytket`, that work together to implement the QFT.

The `QFT` function can be used to create the QFT. It takes the following arguments:<br>
- `n`: number of qubits to use in the QFT circuit<br>
- `arbZZ`: specify whether to use the arbitrary-angle ZZ gate or not, `True`/`False`, default: `False`<br>
- `approx`: if set to integer `k`, then controlled rotations by angles less than $\frac{\pi}{2}^{k}$ do not occur

**Note:** In many presentations of the QFT, the circuit includes a round of SWAP gates at the end of the circuit that reverses the order of the qubits. The QFT circuits in this tutorial do not include this final SWAP step.

**Note:** In `pytket` the $RZZ$ gate is implemented with the $ZZPhase$ circuit function.

In [1]:
import numpy as np
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

In [2]:
def control_phase(circ, theta, q0, q1, arbZZ=False):
    """circuit gadget for performing controlled-[1 0;0 e^i theta]
    Inputs:
        circ: pytket Circuit object
        theta: Z rotation angle (in units of pi)
        q0: control qubit
        q1: target qubit
        arbZZ (bool): enables arbitrary angle RZZ gate
    """
    if arbZZ == False:
        # decompose into CNOTs
        circ.Rz(theta / 2, q1)
        circ.CX(q0, q1)
        circ.Rz(-theta / 2, q1)
        circ.CX(q0, q1)
        circ.Rz(theta / 2, q0)
    elif arbZZ == True:
        circ.Rz(theta / 2, q0)
        circ.Rz(theta / 2, q1)
        circ.ZZPhase(-theta / 2, q0, q1)

In [3]:
def QFT(n, **kwargs):
    """
    Function to implement the Quantum Fourier Transform
    n : number of qubits
    approx: if set to integer k, then sets that largest
                 value of pi/2**k occuring in controlled rotation
    returns circ: pytket Circuit object
    """

    # optional keyword arguments
    arbZZ = kwargs.get("arbZZ", False)
    approx = kwargs.get("approx", None)

    # initialize
    circ_name = "QFT-arbZZ" if arbZZ else "QFT-fixed"
    circ = Circuit(n, n, name=circ_name)
    for j in range(n - 1):
        q = n - 1 - j
        circ.H(q)
        for i in range(j + 1):
            if approx == None or approx >= j + 1 - i:
                control_phase(
                    circ, 1 / (2 ** (j + 1 - i)), q - 1, n - 1 - i, arbZZ=arbZZ
                )
    circ.H(0)
    return circ

## QFT with Fixed Angle Gates

First, create the circuit with fixed-angle gates.

In [4]:
n_qubits = 12

In [5]:
qft_fixed = QFT(n_qubits, arbZZ=False)

In [6]:
render_circuit_jupyter(qft_fixed)

## QFT with Arbitrary Angle ZZ Gates

Second, create the circuit with arbitrary-angle ZZ gates.

In [7]:
qft_arbZZ = QFT(n_qubits, arbZZ=True)

In [8]:
render_circuit_jupyter(qft_arbZZ)

## Compare Results

Now we compare the results of the QFT circuits with and without use of the arbitrary-angle ZZ gates on hardware.

### State Fidelity

The QFT circuit applied to the computational basis state $|x\rangle$ creates the state

\begin{align}<br>
QFT|x\rangle&=\frac{1}{\sqrt{d}}\sum_{y=0}^{d-1} e^{2\pi i x y/d} |y\rangle\\<br>
&= \bigotimes_{j=0}^{n-1}\frac{1}{\sqrt{2}}\sum_{y_j=0}^1e^{2\pi i x 2^j y_j/d}|y_j\rangle\\<br>
&= \bigotimes_{j=0}^{n-1}\frac{1}{\sqrt{2}}\big(|0\rangle+e^{2\pi i x 2^j /d}|1\rangle\big)<br>
\end{align}

where $d=2^n$. Note that this state is unentangled. Therefore the state fidelity can be measured by applying only single-qubit gates to map the state back to the computational basis. In the example circuits above, the initial state $|x\rangle=|0\rangle$, and so the output state is

$$\bigotimes_{j=0}^{n-1}\frac{1}{\sqrt{2}}\big(|0\rangle + |1\rangle\big) = |+\rangle^{\otimes n}$$

The state fidelity can then be measured by applying a Hadamard gate to each qubit and recording the probability of measuring $|0\rangle$.

We define a function to measure all qubits in the Hadamard basis and append this circuit to the QFT circuits:

In [9]:
def meas_Had_basis(orig_circ, n_qubits):
    circ = orig_circ.copy()
    for j in range(n_qubits):
        circ.H(j)
    circ.add_barrier(range(n_qubits))
    circ.measure_all()
    return circ

In [10]:
qft_fid_fixed = meas_Had_basis(qft_fixed, n_qubits)
render_circuit_jupyter(qft_fid_fixed)

In [11]:
qft_fid_arbZZ = meas_Had_basis(qft_arbZZ, n_qubits)
render_circuit_jupyter(qft_fid_arbZZ)

### Define Nexus Configuration

Use qnexus to retrieve an existing project from the database (`arb-angle-demonstration`). The `qnexus.QuantinuumConfig` targets the `H2-Emulator`, hosted on nexus.

In [12]:
import datetime
import qnexus

quantinuum_config = qnexus.QuantinuumConfig(device_name="H2-Emulator")
job_name_suffix = datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")

In [13]:
project = qnexus.projects.get_or_create(name="arb-angle-demonstration")
token = qnexus.context.set_active_project(project)

### Circuit Compilation

In [14]:
ref_fixed = qnexus.circuits.upload(qft_fid_fixed, name="Fixed")
ref_arbzz = qnexus.circuits.upload(qft_fid_arbZZ, name="ArbZZ")

Compile the circuits to use H-Series native gates.

In [15]:
ref_compile_job_arbzz = qnexus.start_compile_job(
    circuits=[ref_arbzz],
    backend_config=quantinuum_config,
    name=f"nexus-compilation-job-arbzz-{job_name_suffix}",
    optimisation_level=1
)

In [16]:
qnexus.jobs.wait_for(ref_compile_job_arbzz)
compiled_circuit_ref_arbzz = qnexus.jobs.results(ref_compile_job_arbzz)[0].get_output()

In [17]:
compiled_circuit_arbzz = compiled_circuit_ref_arbzz.download_circuit()

In [18]:
ref_compile_job_fixed = qnexus.start_compile_job(
    circuits=[ref_fixed],
    backend_config=quantinuum_config,
    name=f"nexus-compilation-job-fixed-{job_name_suffix}",
    optimisation_level=1
)

In [19]:
qnexus.jobs.wait_for(ref_compile_job_fixed);
compiled_circuit_ref_fixed = qnexus.jobs.results(ref_compile_job_fixed)[0].get_output()

In [20]:
compiled_circuit_fixed = compiled_circuit_ref_fixed.download_circuit()

### Circuit Depth and Two-Qubit Gates

Note that the circuit depth number of two-qubit gates for the fixed-angle vs. arbitrary angle is less. The difference increases as more qubits are used.

In [21]:
print("Circuit Depth for fixed-angle QFT:", compiled_circuit_fixed.depth())
print("Circuit Depth for arbitrary-angle QFT:", compiled_circuit_arbzz.depth())
print("Circuit Depth Difference:", compiled_circuit_fixed.depth() - compiled_circuit_arbzz.depth())
print("Number of two-qubit gates for fixed-angle QFT:", compiled_circuit_fixed.n_2qb_gates())
print("Number of two-qubit gates for arbitrary-angle QFT:", compiled_circuit_arbzz.n_2qb_gates())
print("Number of two-qubit gates Difference:", compiled_circuit_fixed.n_2qb_gates() - compiled_circuit_arbzz.n_2qb_gates())

Circuit Depth for fixed-angle QFT: 77
Circuit Depth for arbitrary-angle QFT: 25
Circuit Depth Difference: 52
Number of two-qubit gates for fixed-angle QFT: 132
Number of two-qubit gates for arbitrary-angle QFT: 66
Number of two-qubit gates Difference: 66


### Run the Circuit

Now run the circuits on Quantinuum systems. First compiling the circuits to the backend, then submitting to the device.

In [23]:
n_shots = 100
ref_execute_job_fixed = qnexus.start_execute_job(
    circuits=[compiled_circuit_ref_fixed],
    n_shots=[n_shots],
    backend_config=quantinuum_config,
    name=f"execution-job-fixed-{job_name_suffix}"
)

In [34]:
qnexus.jobs.status(ref_execute_job_fixed)

JobStatus(status=<StatusEnum.RUNNING: 'Circuit is running.'>, message='The job is running.', error_detail=None, completed_time=None, queued_time=None, submitted_time=datetime.datetime(2024, 7, 19, 18, 16, 41, 395596, tzinfo=datetime.timezone.utc), running_time=datetime.datetime(2024, 7, 19, 18, 18, 49, 345604, tzinfo=datetime.timezone.utc), cancelled_time=None, error_time=None, queue_position=None)

In [28]:
ref_execute_job_arbzz = qnexus.start_execute_job(
    circuits=[compiled_circuit_ref_arbzz],
    n_shots=[n_shots],
    backend_config=quantinuum_config,
    name=f"nexus-execution-job-arbzz-{job_name_suffix}"
)

In [35]:
qnexus.jobs.status(ref_execute_job_arbzz)

JobStatus(status=<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>, message='Job has been submitted to Nexus.', error_detail=None, completed_time=None, queued_time=None, submitted_time=datetime.datetime(2024, 7, 19, 18, 17, 56, 781795, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

### Retrieve Results

In [ ]:
qft_fid_fixed_compiled_result = qnexus.jobs.results(ref_execute_job_fixed)[0].get_output().download_result()
qft_fid_arbZZ_compiled_result = qnexus.jobs.results(ref_execute_job_fixed)[0].get_output().download_result()

### Analyze Results

Here the distribution of bitstrings is retrieved to inspect.

In [ ]:
qft_fid_fixed_compiled_distro = qft_fid_fixed_compiled_result.get_distribution()
qft_fid_arbZZ_compiled_distro = qft_fid_arbZZ_compiled_result.get_distribution()

For the QFT with the appended measurement in the Hadamard basis, we expect the QFT to return all 0's in the result bitstring. Investigating the results for both the fixed and arbitrary ZZ versions of QFT, we see this is the bitstring with the highest frequency. This is good, this is what is desired.

In [ ]:
qft_fid_fixed_compiled_distro

In [ ]:
qft_fid_arbZZ_compiled_distro

Comparing the results between the fixed and arbitrary ZZ versions we see that the fidelity is higher using the arbitrary ZZ gate.

In [ ]:
print(
    "Fixed angle QFT:",
    qft_fid_fixed_compiled_distro[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)],
)
print(
    "Arbitrary Angle ZZ QFT:",
    qft_fid_arbZZ_compiled_distro[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)],
)

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>